# Project Notebook

## Social Data Science - exam notebook


#### Authors: 
#### Anton Maach-Møller
#### Christian Nøjgaard Fogdal

## 1. Installations

In [138]:
# pip install geopy
# pip install textblob
# pip install vaderSentiment
# pip install emoji

## 2. Import of used packages

In [190]:
# import modules
import numpy as np
import pandas as pd
import tweepy as tw #package to handle twitter data
import webbrowser
from geopy.geocoders import Nominatim #Geodata for datacleaning states
from geopy.exc import GeocoderTimedOut #Geodata for datacleaning states
from geopy.extra.rate_limiter import RateLimiter #rate limiter to geopy requests
from tqdm import tqdm #progress bar, use code tqdm.pandas()
from time import sleep #module to control speed of extraction
import re #regex
import nltk #Natural Language Toolkit
import random #randomizer
from textblob import TextBlob #To analyse sentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer #Sentiment analyser


## 3. Extraction of data from Twitter

In [174]:

# your Twitter API key and API secret
#my_api_key = "X6oaPMBWYauiGFe9Z9KeCW5HN"
#my_api_secret = "UCvW1Bb3CkXruyVbdhO1CDE1x2kvsrsWEBhA6mk8Mrn7fTF9Ku"
MY_BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAANcAaQEAAAAA84hzZvsLo8rITqxhQJoQHVAd5wI%3DfD5dXkdwihlc4K08dNi3y84D4lVhjiOIREITen9vVe2UVYxFh7"
client = tw.Client(bearer_token=MY_BEARER_TOKEN)

# authenticate
#auth = tw.OAuthHandler(my_api_key, my_api_secret)
#api = tw.API(auth, wait_on_rate_limit=True)

# query to search for tweets
query = "#vaccine lang:en -is:retweet  place_country:US"
# your start and end time for fetching tweets
start_time = "2020-11-10T00:00:00Z"
end_time = "2021-01-05T00:00:00Z"
# get tweets from the API
tweets = client.search_all_tweets(   query=query,
                                     start_time=start_time,
                                     end_time=end_time,
                                     tweet_fields = ["created_at", 
                                                     "text", 
                                                     "source", 
                                                     #"public_metrics"
                                                    ],
                                     user_fields = ["name", "username", "location", "verified", "description"],
                                     place_fields=['place_type', 'country', 'country_code'],
                                     max_results = 10,
                                     #sort_order = ,
                                     expansions=['author_id', 'geo.place_id'],
                                     )

# tweet specific info
print(len(tweets.data))
# user specific info
print(len(tweets.includes["users"]))

# first tweet
first_tweet = tweets.data[0]
dict(first_tweet)

# user information for the first tweet
first_tweet_user = tweets.includes["users"][1]
dict(first_tweet_user)

first_tweet_place = tweets.includes['places'][1]
dict(first_tweet_place)


print("Total Tweets fetched:", len(tweets))

10
10
Total Tweets fetched: 4


## 4. Creating dataframe from extracted data

In [175]:
# create a list of records
tweet_info_ls = []
# iterate over each tweet and corresponding user details
for tweet, user, place in zip(tweets.data, tweets.includes['users'], tweets.includes['places']):
    tweet_info = {
        'created_at': tweet.created_at,
        'text': tweet.text,
        'source': tweet.source,
        #'public_metrics': tweet.public_metrics,
        #'organic_metrics': tweet.organic_metrics,
        'name': user.name,
        'username': user.username,
        'user_location': user.location,
        'tweet_location': place.full_name,
        'country_code': place.country_code,
        #'country': user.country,
        'verified': user.verified,
        'description': user.description
    }
    tweet_info_ls.append(tweet_info)
# create dataframe from the extracted records
tweets_df = pd.DataFrame(tweet_info_ls)
# display the dataframe
tweets_df

,created_at,text,source,name,username,user_location,tweet_location,country_code,verified,description
0,2021-01-04 23:42:58+00:00,Elderly people sitting on sidewalks in Florida...,Twitter for iPhone,Jeff Wolfe 🇺🇦,wolfejeff,"Atlanta, GA","Georgia, USA",US,False,Living in the Atlanta area. Opinions and photo...
1,2021-01-04 23:35:40+00:00,If the government used the same distribution s...,Twitter for Android,Lemme Doodis,LemmeDoodis,"New Hampshire, USA","New Hampshire, USA",US,False,Agnoiologist.
2,2021-01-04 23:27:20+00:00,@anya1anya I believe #educators should be one ...,Twitter for iPhone,🄹🄰🅂🄾🄽 🄻🅄🄺🄴,RunNPelo4Pizza,"Owego, NY","Owego, NY",US,False,Lead Technology Learner at https://t.co/316F3E...
3,2021-01-04 23:03:16+00:00,"@nytimes 2020 just won’t go away, will it? 🙄🤷🏼...",Twitter for iPad,Scott Maier,samaier,"San Antonio, TX","San Francisco, CA",US,False,"Freelancer. Former UC San Francisco, Texas A&M..."
4,2021-01-04 22:57:09+00:00,I believe there’s such thing as the National G...,Twitter for iPhone,Phil McNamara,philindc,"Washington, DC","Washington, DC",US,False,Proudly serving in Biden Admin @usdot. Former ...
5,2021-01-04 22:33:22+00:00,"Do you know someone in #mecklenburg County, #...",Twitter for Android,Jamie,ncrainbowgrrl,"Carrboro, NC","Carrboro, NC",US,False,"human with a conscience, #BanAssaultWeapons, ..."
6,2021-01-04 22:07:30+00:00,Want to know how to make any amount trust that...,Twitter for iPhone,Dr. Rashawn Ray,SociologistRay,"Washington, DC","Mitchellville, MD",US,True,Sociology Professor • @BrookingsInst Senior Fe...
7,2021-01-04 20:06:38+00:00,Received my first of two #COVID19 #vaccine tod...,Twitter for iPhone,Lawyer Pete Schulte,AttyPeteSchulte,"Denver, CO","Desoto, TX",US,False,CO/TX/WY Lawyer. Public Safety Mgr @AuroraGov ...
8,2021-01-04 19:49:40+00:00,The roll-out for the #vaccine to combat the #C...,Twitter for Android,Dr. Melvin G. Brown,GRevmgbrown4,USA,"Silver Spring, MD",US,False,Pastor - Greater New Hope Baptist ChurchDC; Le...
9,2021-01-04 18:12:02+00:00,#Vaccine time!!! Pfizer dose one today! 💉💉💉💉 (...,Foursquare,Michael MPH 🇺🇸 ✌🏼,PhillyPartTwo,PA Congressional District 04,"Pennsylvania, USA",US,False,"Writer, marketer, broadcaster, Philadelphian, ..."


## 5. Cleaning location variable and extracting geo data 

In [176]:
def cleaner(document_location):
    try:
        #document_location = document_location.lower() #To lower case
        document_location = re.sub(r'@[^\s]+','',document_location) #removes @user
        document_location = re.sub(r'#[^\s]+','',document_location) #removes #hashtag
        #document_location = re.sub(r'[\d-]','',document_location) #removes digits
        #document_location = re.sub(r'[^\w\s]','', document_location) #Remove non-alphanumeric characters
        return document_location
    except:
        ("Some variable is None")

tweets_df['user_location'] = tweets_df['user_location'].apply(cleaner)
tweets_df

,created_at,text,source,name,username,user_location,tweet_location,country_code,verified,description
0,2021-01-04 23:42:58+00:00,Elderly people sitting on sidewalks in Florida...,Twitter for iPhone,Jeff Wolfe 🇺🇦,wolfejeff,"Atlanta, GA","Georgia, USA",US,False,Living in the Atlanta area. Opinions and photo...
1,2021-01-04 23:35:40+00:00,If the government used the same distribution s...,Twitter for Android,Lemme Doodis,LemmeDoodis,"New Hampshire, USA","New Hampshire, USA",US,False,Agnoiologist.
2,2021-01-04 23:27:20+00:00,@anya1anya I believe #educators should be one ...,Twitter for iPhone,🄹🄰🅂🄾🄽 🄻🅄🄺🄴,RunNPelo4Pizza,"Owego, NY","Owego, NY",US,False,Lead Technology Learner at https://t.co/316F3E...
3,2021-01-04 23:03:16+00:00,"@nytimes 2020 just won’t go away, will it? 🙄🤷🏼...",Twitter for iPad,Scott Maier,samaier,"San Antonio, TX","San Francisco, CA",US,False,"Freelancer. Former UC San Francisco, Texas A&M..."
4,2021-01-04 22:57:09+00:00,I believe there’s such thing as the National G...,Twitter for iPhone,Phil McNamara,philindc,"Washington, DC","Washington, DC",US,False,Proudly serving in Biden Admin @usdot. Former ...
5,2021-01-04 22:33:22+00:00,"Do you know someone in #mecklenburg County, #...",Twitter for Android,Jamie,ncrainbowgrrl,"Carrboro, NC","Carrboro, NC",US,False,"human with a conscience, #BanAssaultWeapons, ..."
6,2021-01-04 22:07:30+00:00,Want to know how to make any amount trust that...,Twitter for iPhone,Dr. Rashawn Ray,SociologistRay,"Washington, DC","Mitchellville, MD",US,True,Sociology Professor • @BrookingsInst Senior Fe...
7,2021-01-04 20:06:38+00:00,Received my first of two #COVID19 #vaccine tod...,Twitter for iPhone,Lawyer Pete Schulte,AttyPeteSchulte,"Denver, CO","Desoto, TX",US,False,CO/TX/WY Lawyer. Public Safety Mgr @AuroraGov ...
8,2021-01-04 19:49:40+00:00,The roll-out for the #vaccine to combat the #C...,Twitter for Android,Dr. Melvin G. Brown,GRevmgbrown4,USA,"Silver Spring, MD",US,False,Pastor - Greater New Hope Baptist ChurchDC; Le...
9,2021-01-04 18:12:02+00:00,#Vaccine time!!! Pfizer dose one today! 💉💉💉💉 (...,Foursquare,Michael MPH 🇺🇸 ✌🏼,PhillyPartTwo,PA Congressional District 04,"Pennsylvania, USA",US,False,"Writer, marketer, broadcaster, Philadelphian, ..."


In [177]:
geolocator = Nominatim(user_agent="geoapiExercises")

def city_state_country(row):
    coord = f"{row['user_location']}"
    sleep(0.5)
    location = geolocator.geocode(coord,exactly_one=True, addressdetails=True)
    if not location:
        # if you see many in a row, it's probably Nominatim blocking you.
        # if it's just every once in a while, there were just some bad results. 
        print('Failed with location: ', coord)
        row['city'], row['state'], row['country'] = None, None, None
        return row
    address = location.raw['address']
    city = address.get('city', '')
    state = address.get('state', '')
    country = address.get('country', '')
    row['city'] = city
    row['state'] = state
    row['country'] = country
    return row

tweets_df = tweets_df.apply(city_state_country, axis=1)
tweets_df

Failed with location:  PA Congressional District 04


,created_at,text,source,name,username,user_location,tweet_location,country_code,verified,description,city,state,country
0,2021-01-04 23:42:58+00:00,Elderly people sitting on sidewalks in Florida...,Twitter for iPhone,Jeff Wolfe 🇺🇦,wolfejeff,"Atlanta, GA","Georgia, USA",US,False,Living in the Atlanta area. Opinions and photo...,Atlanta,Georgia,United States
1,2021-01-04 23:35:40+00:00,If the government used the same distribution s...,Twitter for Android,Lemme Doodis,LemmeDoodis,"New Hampshire, USA","New Hampshire, USA",US,False,Agnoiologist.,,New Hampshire,United States
2,2021-01-04 23:27:20+00:00,@anya1anya I believe #educators should be one ...,Twitter for iPhone,🄹🄰🅂🄾🄽 🄻🅄🄺🄴,RunNPelo4Pizza,"Owego, NY","Owego, NY",US,False,Lead Technology Learner at https://t.co/316F3E...,,New York,United States
3,2021-01-04 23:03:16+00:00,"@nytimes 2020 just won’t go away, will it? 🙄🤷🏼...",Twitter for iPad,Scott Maier,samaier,"San Antonio, TX","San Francisco, CA",US,False,"Freelancer. Former UC San Francisco, Texas A&M...",San Antonio,Texas,United States
4,2021-01-04 22:57:09+00:00,I believe there’s such thing as the National G...,Twitter for iPhone,Phil McNamara,philindc,"Washington, DC","Washington, DC",US,False,Proudly serving in Biden Admin @usdot. Former ...,Washington,District of Columbia,United States
5,2021-01-04 22:33:22+00:00,"Do you know someone in #mecklenburg County, #...",Twitter for Android,Jamie,ncrainbowgrrl,"Carrboro, NC","Carrboro, NC",US,False,"human with a conscience, #BanAssaultWeapons, ...",,North Carolina,United States
6,2021-01-04 22:07:30+00:00,Want to know how to make any amount trust that...,Twitter for iPhone,Dr. Rashawn Ray,SociologistRay,"Washington, DC","Mitchellville, MD",US,True,Sociology Professor • @BrookingsInst Senior Fe...,Washington,District of Columbia,United States
7,2021-01-04 20:06:38+00:00,Received my first of two #COVID19 #vaccine tod...,Twitter for iPhone,Lawyer Pete Schulte,AttyPeteSchulte,"Denver, CO","Desoto, TX",US,False,CO/TX/WY Lawyer. Public Safety Mgr @AuroraGov ...,Denver,Colorado,United States
8,2021-01-04 19:49:40+00:00,The roll-out for the #vaccine to combat the #C...,Twitter for Android,Dr. Melvin G. Brown,GRevmgbrown4,USA,"Silver Spring, MD",US,False,Pastor - Greater New Hope Baptist ChurchDC; Le...,,,United States
9,2021-01-04 18:12:02+00:00,#Vaccine time!!! Pfizer dose one today! 💉💉💉💉 (...,Foursquare,Michael MPH 🇺🇸 ✌🏼,PhillyPartTwo,PA Congressional District 04,"Pennsylvania, USA",US,False,"Writer, marketer, broadcaster, Philadelphian, ...",None,None,None


In [192]:
# Removes rows, where country != 'United States'.
# This is often due to free text, where the user for example has written 'my bed' or 'the city'
# 'the city' returns "City of London", which is why it is removed.
tweets_df = tweets_df.drop(tweets_df[tweets_df.country != 'United States'].index)

#Empty values to NaN
tweets_df['city'].replace('', np.nan, inplace=True)
tweets_df['state'].replace('', np.nan, inplace=True)

# Also removes rows without state info
tweets_df[~tweets_df['state'].isnull()]
tweets_df.dropna(subset=['state'], inplace=True)
tweets_df

,created_at,text,source,name,username,user_location,tweet_location,country_code,verified,description,city,state,country
0,2021-01-04 23:42:58+00:00,Elderly people sitting on sidewalks in Florida...,Twitter for iPhone,Jeff Wolfe 🇺🇦,wolfejeff,"Atlanta, GA","Georgia, USA",US,False,Living in the Atlanta area. Opinions and photo...,Atlanta,Georgia,United States
1,2021-01-04 23:35:40+00:00,If the government used the same distribution s...,Twitter for Android,Lemme Doodis,LemmeDoodis,"New Hampshire, USA","New Hampshire, USA",US,False,Agnoiologist.,NaN,New Hampshire,United States
2,2021-01-04 23:27:20+00:00,@anya1anya I believe #educators should be one ...,Twitter for iPhone,🄹🄰🅂🄾🄽 🄻🅄🄺🄴,RunNPelo4Pizza,"Owego, NY","Owego, NY",US,False,Lead Technology Learner at https://t.co/316F3E...,NaN,New York,United States
3,2021-01-04 23:03:16+00:00,"@nytimes 2020 just won’t go away, will it? 🙄🤷🏼...",Twitter for iPad,Scott Maier,samaier,"San Antonio, TX","San Francisco, CA",US,False,"Freelancer. Former UC San Francisco, Texas A&M...",San Antonio,Texas,United States
4,2021-01-04 22:57:09+00:00,I believe there’s such thing as the National G...,Twitter for iPhone,Phil McNamara,philindc,"Washington, DC","Washington, DC",US,False,Proudly serving in Biden Admin @usdot. Former ...,Washington,District of Columbia,United States
5,2021-01-04 22:33:22+00:00,"Do you know someone in #mecklenburg County, #...",Twitter for Android,Jamie,ncrainbowgrrl,"Carrboro, NC","Carrboro, NC",US,False,"human with a conscience, #BanAssaultWeapons, ...",NaN,North Carolina,United States
6,2021-01-04 22:07:30+00:00,Want to know how to make any amount trust that...,Twitter for iPhone,Dr. Rashawn Ray,SociologistRay,"Washington, DC","Mitchellville, MD",US,True,Sociology Professor • @BrookingsInst Senior Fe...,Washington,District of Columbia,United States
7,2021-01-04 20:06:38+00:00,Received my first of two #COVID19 #vaccine tod...,Twitter for iPhone,Lawyer Pete Schulte,AttyPeteSchulte,"Denver, CO","Desoto, TX",US,False,CO/TX/WY Lawyer. Public Safety Mgr @AuroraGov ...,Denver,Colorado,United States


## 6. Cleaning of the text variable used to analyse sentiment

In [193]:
tweets_df.text[3]

'@nytimes 2020 just won’t go away, will it? 🙄🤷🏼\u200d♂️💉 #COVID #vaccine'

In [199]:
tweets_df2 = tweets_df.copy()
tweets_df2

,created_at,text,source,name,username,user_location,tweet_location,country_code,verified,description,city,state,country
0,2021-01-04 23:42:58+00:00,Elderly people sitting on sidewalks in Florida...,Twitter for iPhone,Jeff Wolfe 🇺🇦,wolfejeff,"Atlanta, GA","Georgia, USA",US,False,Living in the Atlanta area. Opinions and photo...,Atlanta,Georgia,United States
1,2021-01-04 23:35:40+00:00,If the government used the same distribution s...,Twitter for Android,Lemme Doodis,LemmeDoodis,"New Hampshire, USA","New Hampshire, USA",US,False,Agnoiologist.,NaN,New Hampshire,United States
2,2021-01-04 23:27:20+00:00,@anya1anya I believe #educators should be one ...,Twitter for iPhone,🄹🄰🅂🄾🄽 🄻🅄🄺🄴,RunNPelo4Pizza,"Owego, NY","Owego, NY",US,False,Lead Technology Learner at https://t.co/316F3E...,NaN,New York,United States
3,2021-01-04 23:03:16+00:00,"@nytimes 2020 just won’t go away, will it? 🙄🤷🏼...",Twitter for iPad,Scott Maier,samaier,"San Antonio, TX","San Francisco, CA",US,False,"Freelancer. Former UC San Francisco, Texas A&M...",San Antonio,Texas,United States
4,2021-01-04 22:57:09+00:00,I believe there’s such thing as the National G...,Twitter for iPhone,Phil McNamara,philindc,"Washington, DC","Washington, DC",US,False,Proudly serving in Biden Admin @usdot. Former ...,Washington,District of Columbia,United States
5,2021-01-04 22:33:22+00:00,"Do you know someone in #mecklenburg County, #...",Twitter for Android,Jamie,ncrainbowgrrl,"Carrboro, NC","Carrboro, NC",US,False,"human with a conscience, #BanAssaultWeapons, ...",NaN,North Carolina,United States
6,2021-01-04 22:07:30+00:00,Want to know how to make any amount trust that...,Twitter for iPhone,Dr. Rashawn Ray,SociologistRay,"Washington, DC","Mitchellville, MD",US,True,Sociology Professor • @BrookingsInst Senior Fe...,Washington,District of Columbia,United States
7,2021-01-04 20:06:38+00:00,Received my first of two #COVID19 #vaccine tod...,Twitter for iPhone,Lawyer Pete Schulte,AttyPeteSchulte,"Denver, CO","Desoto, TX",US,False,CO/TX/WY Lawyer. Public Safety Mgr @AuroraGov ...,Denver,Colorado,United States


In [202]:

def cleaner(document_tweet):
    """
    Cleaning string variable
    """
    document_tweet = document_tweet.lower() #To lower case
    document_tweet = re.sub(r'@[^\s]+','',document_tweet) #removes @user
    #document_tweet = re.sub(r'#[^\s]+','',document_tweet) #removes # and #hashtagstring
    document_tweet = re.sub(r'[\d-]','',document_tweet) #removes digits
    document_tweet = re.sub(r'https[^\s]+','',document_tweet) #removes links starting with https
    #document_tweet = re.sub(r'[^\w\s]','', document_tweet) #Remove non-alphanumeric characters
    document_tweet = re.sub(r'\r\n|\r|\n', ' ', document_tweet) #replaces newline with with space
    document_tweet = re.sub(r'[#]','',document_tweet) #Removes only #-sign
    return document_tweet

tweets_df2['text'] = tweets_df2['text'].apply(cleaner)
tweets_df2.text[3]

'  just won’t go away, will it? 🙄🤷🏼\u200d♂️💉 covid vaccine'

In [203]:
res = TextBlob(tweets_df2.text[3])
print(res.sentiment.polarity)
############

KOMMET HERTIL

############

0.0


In [188]:
sid_obj= SentimentIntensityAnalyzer()
print(sid_obj.polarity_scores(tweets_df2.text[1]))

{'neg': 0.0, 'neu': 0.925, 'pos': 0.075, 'compound': 0.2732}


In [131]:
def get_tweet_sentiment(self, tweet):
    '''
    Utility function to classify sentiment of passed tweet
    using textblob's sentiment method
    '''
    # create TextBlob object of passed tweet text
    analysis = TextBlob(tweets_df2['text'])
    # set sentiment
    if analysis.sentiment.polarity > 0:
        return 'positive'
    elif analysis.sentiment.polarity == 0:
        return 'neutral'
    else:
        return 'negative'

In [222]:
# Function used to geocode locations and override timeout error
def do_geocode(address):
    geopy = Nominatim(user_agent="Twitter_data")
    try:
        return geopy.geocode(address,exactly_one=True, addressdetails=True)
    except GeocoderTimedOut:
        return do_geocode(address)

# Creating Geocoded Location column
tweets_df['GeocodedLocation']=tweets_df['location'].apply(lambda x: do_geocode(x) if x != None else None)


0    (Washington, District of Columbia, United Stat...
1    (City of New York, New York, United States, (4...
2    (Texas, United States, (31.2638905, -98.5456116))
3    (Bridgewater, Plymouth County, Massachusetts, ...
4    (Annapolis, Anne Arundel County, Maryland, Uni...
5                                                 None
6    (City of London, Greater London, England, Unit...
7    (Tyler, Smith County, Texas, United States, (3...
Name: GeocodedLocation, dtype: object


In [221]:
# Use the geopy locator with Nominatim/OpenStreetMaps
locator = Nominatim(user_agent='myGeocoder')
adress = ['New York City']
location = locator.geocode(adress,exactly_one=True, addressdetails=True)
location.raw


county = (location.raw['address']).get('state')
county

'New York'

In [207]:

geolocator = Nominatim(user_agent="geoapiExercises")

def city_state_country(adress):
    location = geolocator.geocode(adress, exactly_one=True)
    address = location.raw['address']
    state = address.get('state', '')
    country = address.get('country', '')
    return city, state, country


def do_geocode(address):
    geopy = Nominatim(user_agent="Twitter_data")
    try:
        return geopy.geocode(address,exactly_one=True, addressdetails=True)
    except GeocoderTimedOut:
        return do_geocode(address)

# Creating Geocoded Location column

tweets_df['state']=tweets_df['location'].apply(lambda x: do_geocode(x) if x != None else None)
tweets_df




AttributeError: 'NoneType' object has no attribute 'raw'

In [213]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="geoapiExercises")

state1 = "Denmark"
print("State Name:",state1)
location = geolocator.geocode(state1)
print("State Name/Country Name: ")
print(location.address)


State Name: Denmark
State Name/Country Name: 
Danmark


In [192]:
# Function used to geocode locations and override timeout error
def do_geocode(address):
    try:
        geolocator = Nominatim(user_agent="Twitter_location_to_us_state")
        location = geolocator.geocode(adress, addressdetails=True)
        state = location.raw['address']['state']
        return state
        #return geopy.geocode(address,exactly_one=True, addressdetails=True)
    except GeocoderTimedOut:
        return do_geocode(address)

# Creating Geocoded Location column
tweets_df['state']=tweets_df['location'].apply(lambda x: do_geocode(x) if x != None else None)

NameError: name 'adress' is not defined

In [193]:
tweets_df

,created_at,text,source,public_metrics,name,username,location,State_country,country_code,verified,description
0,2020-12-31 23:39:59+00:00,Scientists: We have a #vaccine! \nAMERICANnnnn...,Twitter for Android,None,Captain Co-dependent,TaeX2times,"Washington, DC","Arlington, VA",US,False,🍋🍋🥝🍉🍑 AquaGirl ♒\nHost/ Creator of the Normal...
1,2020-12-31 23:38:46+00:00,The #vaccine rollout is a mess,Twitter for iPhone,None,George Henderson,GeoPHenderson,New York City,"Manhattan, NY",US,False,"Party Planner, Dandy, Man About Town, Former D..."
2,2020-12-31 23:26:10+00:00,"All these ""pro-life"" catholics who refuse to w...",Twitter for Android,None,McRector,lb_rector,Texas,"Texas, USA",US,False,Music lover. Film fanatic. Craft beer enthusia...
3,2020-12-31 22:27:35+00:00,#goodbye2020 #hello2021 #maskup #keephysicaldi...,Instagram,None,"Joe Oravecz, Ph.D.",DrJoeO,"Bridgewater, MA","Bridgewater, MA",US,False,"Vice President: @BridgeStateU ""All About the S..."
4,2020-12-31 21:52:50+00:00,They won't say WHY?! W.T.H.! Do they know some...,Twitter for Android,None,RealMaryZ,RealMaryz,"Annapolis, Maryland","Londontowne, MD",US,False,"The Earth is the Mother of all people, and all..."
5,2020-12-31 20:53:13+00:00,Just in time for the #NewYear!\n\nGrateful for...,Twitter for iPhone,None,"aL'Ai alvarez, MD",alvarezzzy,#FirstGenMD at Stanford,"San Francisco, CA",US,False,"Well-Being Director|Co-Chair, Human Potential ..."
6,2020-12-31 20:46:31+00:00,Look.\n\nEverybody knows that the vaccine is a...,Twitter for Android,None,🇺🇦(((The Dividist))) 🇺🇦,Dividist,The City,"Tyler, TX",US,False,"Social liberal, fiscal conservative, civil lib..."
7,2020-12-31 20:42:16+00:00,Just got my first #shot of the #vaccine!,Twitter for iPhone,None,Paul Buchanan,celadon100,"Tyler, Texas, USA","Chicago, IL",US,False,I am a geochemist who does a bit of research a...


In [102]:
tweets_df.GeocodedLocation[1]

0                                                 None
1    (Las Vegas, Clark County, Nevada, United State...
2    (Plymouth, Hennepin County, Minnesota, United ...
3    (Orlando, Orange County, Florida, United State...
4    (Brooklyn, Kings County, City of New York, New...
5    (City of New York, New York, United States, (4...
6    (Fort Lauderdale, Broward County, Florida, Uni...
7    (Boston, Suffolk County, Massachusetts, United...
8    (Memphis, Shelby County, West Tennessee, Tenne...
Name: GeocodedLocation, dtype: object

In [182]:
geolocator = Nominatim(user_agent="dasdasda")
tlocation = geolocator.geocode("Fort Lauderdale")
print(location.address.state)


AttributeError: 'str' object has no attribute 'state'

In [198]:
from geopy.geocoders import Nominatim

# address is a String e.g. 'Berlin, Germany'
# addressdetails=True does the magic and gives you also the details
location = geolocator.geocode("#FirstGenMD at Stanford", addressdetails=True)

location.raw['address']

AttributeError: 'NoneType' object has no attribute 'raw'

In [143]:
class location

SyntaxError: invalid syntax (952703290.py, line 1)

In [148]:
people = {'adress': {'state': 'Minnesota', 'age': '27', 'sex': 'Male'},
          2: {'name': 'Marie', 'age': '22', 'sex': 'Female'}}

print(people['adress']['state'])
#print(people[1]['age'])
#print(people[1]['sex'])

Minnesota


In [149]:
print(location['address']['state'])



TypeError: tuple indices must be integers or slices, not str

In [86]:
tweets_df.location.address

AttributeError: 'Series' object has no attribute 'address'

In [120]:
tweets_df.GeocodedLocation[1][0]

'Las Vegas, Clark County, Nevada, United States'

In [72]:
# Create the Latitude Column
country=[]
for i in tweets_df['GeocodedLocation']:
    if i== None:
        lat.append(None)
    else:
        lat.append(i.latitude)
tweets_df['Latitude']=lat
tweets_df['Latitude'].astype('float')

# Create the Longitude Column
long=[]
for i in tweets_df['GeocodedLocation']:
    if i== None:
        long.append(None)
    else:
        long.append(i.longitude)
tweets_df['Longitude']=long
tweets_df['Longitude'].astype('float')

0                                                 None
1    (Las Vegas, Clark County, Nevada, United State...
2    (Plymouth, Hennepin County, Minnesota, United ...
3    (Orlando, Orange County, Florida, United State...
4    (Brooklyn, Kings County, City of New York, New...
5    (City of New York, New York, United States, (4...
6    (Fort Lauderdale, Broward County, Florida, Uni...
7    (Boston, Suffolk County, Massachusetts, United...
8    (Memphis, Shelby County, West Tennessee, Tenne...
Name: GeocodedLocation, dtype: object

In [62]:
# Ref: https://geopy.readthedocs.io/en/stable/#usage-with-pandas
geolocator = Nominatim(user_agent="Twitter_data")

#geocode = RateLimiter(geolocator.geocode, min_delay_seconds=3, max_retries=5)
#tweets_df["location"] = tweets_df["location"].progress_apply(geocode, language="en") # Some locations are in hindi, chinese. Language=’en’ returns location in english
#tweets_df["coordinates"] = tweets_df["location"].apply(lambda loc: tuple(loc.point) if loc else None)
tweets_df["state"] = tweets_df["location"].apply(lambda loc: loc[0].split(',')[0] if loc else None)
#tweets_df["country"] = tweets_df["location"].apply(lambda loc: loc[0].split(',')[-1] if loc else None)
tweets_df

,created_at,text,source,public_metrics,location,country,country_code,verified,description,GeocodedLocation,state
0,2022-08-10 22:46:23+00:00,Can we utilize the absorbable micro needle pat...,Twitter for iPhone,None,"Minneapolis/St. Paul, MN",M,US,False,"MPH, CIC. Vaccine and Infectious Disease Geek....",None,Mi
1,2022-08-10 22:34:33+00:00,A combined #influenza and #COVID19 mRNA #vacci...,Twitter for Android,None,"Las Vegas, NV",L,US,False,Future of Healthcare & Life Sciences | Science...,"(Las Vegas, Clark County, Nevada, United State...",La
2,2022-08-10 21:04:18+00:00,@michren03 Twitter blocked me after posting th...,Twitter for Android,None,"Plymouth, MN",P,US,False,Freelance Photographer/Photojournalist \n📷 @ca...,"(Plymouth, Hennepin County, Minnesota, United ...",Pl
3,2022-08-10 19:44:28+00:00,To all the “AUTHENTIC/Fake #DL men please go g...,Twitter for iPhone,None,"Orlando, FL",O,US,False,35 Black Male Anything else feel free to ask!!...,"(Orlando, Orange County, Florida, United State...",Or
4,2022-08-10 15:56:16+00:00,Here are some recommended immunizations for ch...,Twitter for iPhone,None,"Brooklyn, NY",B,US,False,"COPO is a full-service nonprofit in Brooklyn, ...","(Brooklyn, Kings County, City of New York, New...",Br
5,2022-08-10 12:45:49+00:00,"All I saw was “sneaky link, who up, come suck ...",Twitter for iPhone,None,New York City,N,US,True,Reporter @CBSNewYork,"(City of New York, New York, United States, (4...",Ne
6,2022-08-10 11:11:56+00:00,FDA issues emergency use authorization for new...,Twitter for iPhone,None,Fort Lauderdale,F,US,False,Lover of life,"(Fort Lauderdale, Broward County, Florida, Uni...",Fo
7,2022-08-10 00:02:57+00:00,Got this notice that my appointment to get my ...,Instagram,None,"Boston, MA",B,US,False,The Official Twitter Account-𝐃𝐚𝐧 𝐁𝐚𝐫𝐨𝐮𝐜𝐡 𝐋𝐚𝐛 a...,"(Boston, Suffolk County, Massachusetts, United...",Bo
8,2022-08-09 15:12:09+00:00,📣 Fresh from the press 📑 @scilmmunology\n\nChe...,Twitter for iPhone,None,"Memphis, TN",M,US,False,Assistant Professor Department of Community & ...,"(Memphis, Shelby County, West Tennessee, Tenne...",Me


In [108]:
# import the pandas library

# create a list of records
tweet_info_ls2 = []
# iterate over each tweet and corresponding user details
for tweet, user in zip(tweets_df, tweets.includes['places']):
    tweet_info2 = {
        'created_at': tweets_df.created_at,
        'text': tweets_df.text,
        'source': tweets_df.source,
        'name': tweets_df.name,
        'username': tweets_df.username,
        'location': tweets_df.location,
        'country': place.full_name,
        'verified': tweets_df.verified,
        'description': tweets_df.description
    }
    tweet_info_ls2.append(tweet_info2)
# create dataframe from the extracted records
tweets_df2 = pd.DataFrame(tweet_info_ls2)
# display the dataframe
tweets_df2


,created_at,text,source,name,username,location,country,verified,description
0,0 2021-12-08 20:38:34+00:00 1 2021-12-08 2...,0 En dag som i dag vil jeg ønske at jeg boe...,0 Twitter for Android 1 Twitter for Andr...,0 Bjarni Klein Olsen 🇫🇴 1...,0 KleinOlsen 1 evafogdk 2 ...,"0 Føroyar, Streimoy 1 ...","0 Vordingborg, Danmark 1 Vordingborg, Da...",0 False 1 False 2 False 3 False 4 ...,"0 Hjemvendt færing, tilbage på øerne efter ..."
1,0 2021-12-08 20:38:34+00:00 1 2021-12-08 2...,0 En dag som i dag vil jeg ønske at jeg boe...,0 Twitter for Android 1 Twitter for Andr...,0 Bjarni Klein Olsen 🇫🇴 1...,0 KleinOlsen 1 evafogdk 2 ...,"0 Føroyar, Streimoy 1 ...","0 Vordingborg, Danmark 1 Vordingborg, Da...",0 False 1 False 2 False 3 False 4 ...,"0 Hjemvendt færing, tilbage på øerne efter ..."
2,0 2021-12-08 20:38:34+00:00 1 2021-12-08 2...,0 En dag som i dag vil jeg ønske at jeg boe...,0 Twitter for Android 1 Twitter for Andr...,0 Bjarni Klein Olsen 🇫🇴 1...,0 KleinOlsen 1 evafogdk 2 ...,"0 Føroyar, Streimoy 1 ...","0 Vordingborg, Danmark 1 Vordingborg, Da...",0 False 1 False 2 False 3 False 4 ...,"0 Hjemvendt færing, tilbage på øerne efter ..."
3,0 2021-12-08 20:38:34+00:00 1 2021-12-08 2...,0 En dag som i dag vil jeg ønske at jeg boe...,0 Twitter for Android 1 Twitter for Andr...,0 Bjarni Klein Olsen 🇫🇴 1...,0 KleinOlsen 1 evafogdk 2 ...,"0 Føroyar, Streimoy 1 ...","0 Vordingborg, Danmark 1 Vordingborg, Da...",0 False 1 False 2 False 3 False 4 ...,"0 Hjemvendt færing, tilbage på øerne efter ..."
4,0 2021-12-08 20:38:34+00:00 1 2021-12-08 2...,0 En dag som i dag vil jeg ønske at jeg boe...,0 Twitter for Android 1 Twitter for Andr...,0 Bjarni Klein Olsen 🇫🇴 1...,0 KleinOlsen 1 evafogdk 2 ...,"0 Føroyar, Streimoy 1 ...","0 Vordingborg, Danmark 1 Vordingborg, Da...",0 False 1 False 2 False 3 False 4 ...,"0 Hjemvendt færing, tilbage på øerne efter ..."
5,0 2021-12-08 20:38:34+00:00 1 2021-12-08 2...,0 En dag som i dag vil jeg ønske at jeg boe...,0 Twitter for Android 1 Twitter for Andr...,0 Bjarni Klein Olsen 🇫🇴 1...,0 KleinOlsen 1 evafogdk 2 ...,"0 Føroyar, Streimoy 1 ...","0 Vordingborg, Danmark 1 Vordingborg, Da...",0 False 1 False 2 False 3 False 4 ...,"0 Hjemvendt færing, tilbage på øerne efter ..."


In [ ]:
import tweepy

client = tweepy.Client(bearer_token=MY_BEARER_TOKEN)

# Replace with your own search query
query = 'covid -is:retweet place_country:DK'

tweets = client.search_all_tweets(query=query, 
                                  tweet_fields=['context_annotations', 'created_at', 'geo'],
                                  place_fields=['place_type', 'country'], 
                                  expansions='geo.place_id', max_results=10)

# Get list of places from includes object
places = {p["id"]: p for p in tweets.includes['places']}

for tweet in tweets.data:
    print(tweet.id)
    print(tweet.text)
    if places[tweet.geo['place_id']]:
        place = places[tweet.geo['place_id']]
        print(place.full_name)
        
        

1557836753897230336
@JohnH1337 @stinuslindgreen Nej pesticider, PFAS, PFOS osv kan ikke give Covid, men det kan gøre dig mindre modstandsdygtig over for sygdom inkl. C. Tænker ikke at man bliver syg af et glas, men over tid bliver det til meget + blandet med hvor vi ellers udsætters for det og så er vi oppe på store mængder ?
København, Danmark
1557662964148326402
@stinuslindgreen Det meste af mit voksne liv har jeg råbt op om klimaforandring, været glad for min stofpose osv.. Der udledes ekstrem meget kemi ud i vores grundvand og direkte ud i vores haner, kemi der under Covid har lært os, om du spiser/drikker det, ifører dig det eller indånder det, så r👇🏻
København, Danmark
1557090026336538626
@mikemwilson @Lil_Pedro4Lyfe Wait… a new covid variant that is…. Er… undetectable? I’ve had fluish symps for two months on/off, mostly on.
København, Danmark
1557022011007852544
@anders_hviid @NatureComms Is COVID-19 one disease or different diseases related to different strains of the virus?
Vo